In [4]:
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python

     |████████████████████████████████| 52.4 MB 17.9 MB/s eta 0:00:01�▍                   | 20.2 MB 9.7 MB/s eta 0:00:04     |███████████████████████▊        | 38.9 MB 9.7 MB/s eta 0:00:02


In [28]:
import pandas as pd
import os
import shutil 
import os.path
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import numpy as np
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
from pathlib import Path

import cv2
import random
from PIL import Image, ImageEnhance
from pathlib import Path
import matplotlib.pyplot as plt
import uuid
import os
import shutil

In [16]:
cnn = tf.keras.models.load_model('./model/my_model')

In [23]:
CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath) 
    res = cv2.resize(img_array, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
#     return np.reshape(res, (3, IMG_SIZE, IMG_SIZE))
    return res

path, dirs, files = next(os.walk("./data/unlabeled"))
file_count = len(files)

print(file_count)

l = os.listdir("./data/unlabeled")
random.shuffle(l)
count = 0
for file in l:
    count +=1
    if (count % 50 == 0):
        print(count)
    if (file != ".ipynb_checkpoints"):
        image = prepare('./data/unlabeled/' + str(file)).tolist()
        #print(image.shape)
        prediction = cnn.predict([image])
        if (prediction[0][0] < prediction[0][1]):
            predicted_result = "non-melanoma"
            try:
                shutil.copy('./data/unlabeled/'+file, './predict_images/'+file[:-4]+"_0.jpg")
            except:
                print("FAILED")
        else:
            predicted_result = "melanoma"
            try:
                shutil.copy('./data/unlabeled/'+file, './predict_images/'+file[:-4]+"_1.jpg")
            except:
                print("FAILED")

7018
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000


In [14]:
# !mkdir predict_images

In [24]:
path, dirs, files = next(os.walk("./predict_images"))
file_count = len(files)
print("Number of files: " + str(file_count))

train_set = len(files) * 0.6
valid_set = len(files) * 0.4

train_count = 0
validation_count = 0

overall_count = 0
for file in os.listdir("./predict_images"):
    if (file != ".ipynb_checkpoints"):
        if (overall_count < train_set):
            source = './predict_images/' + str(file)
            if (file.endswith('0.jpg')):
                destination = './predict_train/non-melanoma/' + str(file)
            else:
                destination = './predict_train/melanoma/' + str(file)
                
            shutil.copy (source, destination)
            
#             print("Train: " + str(file))
            train_count += 1
        else:
            source1 = './predict_images/' + str(file)
            if (file.endswith('0.jpg')):
                destination1 = './predict_valid/non-melanoma/' + str(file)
            else:
                destination1 = './predict_valid/melanoma/' + str(file)

            shutil.copy (source1, destination1)
#             print("Validation: " + str(file))
            validation_count += 1
    overall_count += 1

print(train_count)
print(validation_count)
print(overall_count)

path, dirs, files = next(os.walk("./predict_valid/melanoma"))
file_count = len(files)
print ("Validation dataset length: " + str(file_count))

path, dirs, files = next(os.walk("./predict_train/melanoma"))
file_count = len(files)
print("Train dataset length: " + str(file_count))

Number of files: 7018
4211
2807
7018
Validation dataset length: 704
Train dataset length: 1084


In [22]:
# shutil.rmtree('./predict_train')
# shutil.rmtree('./predict_valid')

# !mkdir predict_train
# !mkdir predict_valid
# !mkdir predict_train/melanoma
# !mkdir predict_train/non-melanoma
# !mkdir predict_valid/melanoma
# !mkdir predict_valid/non-melanoma

In [27]:
file_paths = {'./train/melanoma/': './predict_train/melanoma/',
              './train/non-melanoma/': './predict_train/non-melanoma/',
              './valid/melanoma/': './predict_valid/melanoma/',
              './valid/non-melanoma/': './predict_valid/non-melanoma/'}
for path in list(file_paths.keys()):
    for file in os.listdir(path):
        shutil.copy(path+file, file_paths[path]+file)

In [32]:
def sharpen(img, factor):
    enhancer_sharpness = ImageEnhance.Sharpness(img)
    return enhancer_sharpness.enhance(factor)

def contrast(img, factor):
    enhancer_contrast = ImageEnhance.Contrast(img)
    return enhancer_contrast.enhance(factor)

def color(img, factor):
    enhancer_color = ImageEnhance.Color(img)
    return enhancer_color.enhance(factor)

def rotate(img, degrees):
    return img.rotate(degrees)

In [33]:
randContrastMin, randContrastMax = (0.8, 1.2)
randSharpenMin, randSharpenMax  = (0.8, 1.2)
randColorMin, randColorMax    = (0.8, 1.2)
multiplier = 2

In [34]:
inputBasePath_t = './predict_train/'
outputBasePath_t  = './predict_train_processed/'
folders = ['melanoma', 'non-melanoma']

In [35]:
inputBasePath_v = './predict_valid/'
outputBasePath_v  = './predict_valid_processed/'

In [30]:
# !mkdir predict_train_processed
# !mkdir predict_valid_processed

# !mkdir predict_train_processed/non-melanoma
# !mkdir predict_train_processed/melanoma
# !mkdir predict_valid_processed/non-melanoma
# !mkdir predict_valid_processed/melanoma

In [36]:
index = 0
count = 0
for f in folders:
    plist = Path(inputBasePath_t + f + '/').glob('*.jpg')

    outpath = outputBasePath_t + f + '/' 
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    
    for path in os.listdir("./predict_train/" + f):
        count += 1
        if (count % 50 == 0):
            print(count)
        
        if (path != ".ipynb_checkpoints"):
            i = Image.open("./predict_train/" + f + "/" + path)
            original_image = i
            path_new = path[:-4]+"1"+".jpg"

            for m in range(5):

                randContrast = random.uniform(randContrastMin, randContrastMax)
                randSharpen = random.uniform(randSharpenMin, randSharpenMax)
                randColor = random.uniform(randColorMin, randColorMax)
                i = contrast(i, randContrast)
                i = sharpen(i, randSharpen)
                i = color(i, randColor)

                i.save(outputBasePath_t + f + '/' + str(m) + path) 
                #print('.', end='')
            original_image.save(outputBasePath_t + f + '/' + path_new)
    index += 1

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250


In [37]:
index = 0
for f in folders:
    plist = Path(inputBasePath_v + f + '/').glob('*.jpg')

    outpath = outputBasePath_v + f + '/' 
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    
    for path in os.listdir("./predict_valid/" + f):
        print(path)
        
        if (path != ".ipynb_checkpoints"):
            i = Image.open("./predict_valid/" + f + "/" + path)
            original_image = i
            path_new = path[:-4]+"1"+".jpg"

            for m in range(5):

                randContrast = random.uniform(randContrastMin, randContrastMax)
                randSharpen = random.uniform(randSharpenMin, randSharpenMax)
                randColor = random.uniform(randColorMin, randColorMax)
                i = contrast(i, randContrast)
                i = sharpen(i, randSharpen)
                i = color(i, randColor)

                i.save(outputBasePath_v + f + '/' + str(m) + path) 
                print('.', end='')
            original_image.save(outputBasePath_v + f + '/' + path_new)
    index += 1

1_1.jpg
.....5127_1.jpg
.....2437_1.jpg
.....5964_1.jpg
.....439_1.jpg
.....1029_1.jpg
.....1897_1.jpg
.....3773_1.jpg
.....1091_1.jpg
.....2773_1.jpg
.....5263_1.jpg
.....2736_1.jpg
.....481_1.jpg
.....5999_1.jpg
.....3858_1.jpg
.....1328_1.jpg
.....6128_1.jpg
.....705_1.jpg
.....2063_1.jpg
.....2865_1.jpg
.....740_1.jpg
.....3820_1.jpg
.....4573_1.jpg
.....3026_1.jpg
.....6640_1.jpg
.....4959_1.jpg
.....6229_1.jpg
.....5959_1.jpg
.....2362_1.jpg
.....404_1.jpg
.....1812_1.jpg
.....5844_1.jpg
.....271_1.jpg
.....2183_1.jpg
.....234_1.jpg
.....4801_1.jpg
.....4493_1.jpg
.....3183_1.jpg
.....519_1.jpg
.....2616_1.jpg
.....135_1.jpg
.....2653_1.jpg
.....4306_1.jpg
.....6833_1.jpg
.....6070_1.jpg
.....4792_1.jpg
.....1525_1.jpg
.....6718_1.jpg
.....67_1.jpg
.....618_1.jpg
.....22_1.jpg
.....6952_1.jpg
.....6154_1.jpg
.....852_1.jpg
.....2777_1.jpg
.....2732_1.jpg
.....485_1.jpg
.....779_1.jpg
.....3819_1.jpg
.....4166_1.jpg
.....6210_1.jpg
.....5123_1.jpg
.....5960_1.jpg
.....6799_1.jpg
.

.....3118_1.jpg
.....4408_1.jpg
.....44_1.jpg
.....5024_1.jpg
.....5867_1.jpg
.....1607_1.jpg
.....3571_1.jpg
.....1642_1.jpg
.....4867_1.jpg
.....1293_1.jpg
.....5709_1.jpg
.....2219_1.jpg
.....2509_1.jpg
.....5019_1.jpg
.....84_1.jpg
.....6407_1.jpg
.....6093_1.jpg
.....5771_1.jpg
.....193_1.jpg
.....5318_1.jpg
.....968_1.jpg
.....3608_1.jpg
.....3160_1.jpg
.....1682_1.jpg
.....3125_1.jpg
.....2923_1.jpg
.....2160_1.jpg
.....2966_1.jpg
.....6975_1.jpg
.....3038_1.jpg
.....6272_1.jpg
.....1762_1.jpg
.....2886_1.jpg
.....2414_1.jpg
.....5104_1.jpg
.....5590_1.jpg
.....6988_1.jpg
.....4629_1.jpg
.....4454_1.jpg
.....6722_1.jpg
.....3144_1.jpg
.....4411_1.jpg
.....622_1.jpg
.....18_1.jpg
.....3669_1.jpg
.....1970_1.jpg
.....4384_1.jpg
.....1176_1.jpg
.....2694_1.jpg
.....2245_1.jpg
.....5755_1.jpg
.....566_1.jpg
.....2568_1.jpg
.....5078_1.jpg
.....5299_1.jpg
.....2789_1.jpg
.....4120_1.jpg
.....6_1.jpg
.....787_1.jpg
.....2430_1.jpg
.....5963_1.jpg
.....2475_1.jpg
.....313_1.jpg
.....55

.....6268_0.jpg
.....189_0.jpg
.....558_0.jpg
.....6458_0.jpg
.....6089_0.jpg
.....1599_0.jpg
.....1665_0.jpg
.....4003_0.jpg
.....6375_0.jpg
.....3513_0.jpg
.....3187_0.jpg
.....4497_0.jpg
.....4046_0.jpg
.....4805_0.jpg
.....3556_0.jpg
.....6330_0.jpg
.....5840_0.jpg
.....2513_0.jpg
.....5046_0.jpg
.....230_0.jpg
.....5805_0.jpg
.....2187_0.jpg
.....5497_0.jpg
.....2939_0.jpg
.....63_0.jpg
.....659_0.jpg
.....1698_0.jpg
.....6759_0.jpg
.....1249_0.jpg
.....6074_0.jpg
.....3612_0.jpg
.....1564_0.jpg
.....6837_0.jpg
.....3657_0.jpg
.....6031_0.jpg
.....6872_0.jpg
.....4347_0.jpg
.....1521_0.jpg
.....4796_0.jpg
.....3286_0.jpg
.....937_0.jpg
.....2612_0.jpg
.....5302_0.jpg
.....174_0.jpg
.....5796_0.jpg
.....2286_0.jpg
.....972_0.jpg
.....131_0.jpg
.....5347_0.jpg
.....2941_0.jpg
.....2102_0.jpg
.....2596_0.jpg
.....5086_0.jpg
.....621_0.jpg
.....2904_0.jpg
.....5457_0.jpg
.....4412_0.jpg
.....1274_0.jpg
.....3102_0.jpg
.....6764_0.jpg
.....4880_0.jpg
.....1231_0.jpg
.....4457_0.jpg
...

.....6215_0.jpg
.....3473_0.jpg
.....4920_0.jpg
.....1481_0.jpg
.....6191_0.jpg
.....6540_0.jpg
.....3326_0.jpg
.....1050_0.jpg
.....6997_0.jpg
.....3363_0.jpg
.....6505_0.jpg
.....4673_0.jpg
.....1015_0.jpg
.....2326_0.jpg
.....440_0.jpg
.....5636_0.jpg
.....2363_0.jpg
.....5673_0.jpg
.....405_0.jpg
.....897_0.jpg
.....2899_0.jpg
.....5958_0.jpg
.....6228_0.jpg
.....3899_0.jpg
.....4537_0.jpg
.....3864_0.jpg
.....6641_0.jpg
.....6290_0.jpg
.....4572_0.jpg
.....1314_0.jpg
.....3062_0.jpg
.....6604_0.jpg
.....390_0.jpg
.....5537_0.jpg
.....741_0.jpg
.....2027_0.jpg
.....704_0.jpg
.....2821_0.jpg
.....2062_0.jpg
.....1439_0.jpg
.....6129_0.jpg
.....2979_0.jpg
.....23_0.jpg
.....619_0.jpg
.....66_0.jpg
.....6877_0.jpg
.....4342_0.jpg
.....1524_0.jpg
.....6034_0.jpg
.....4793_0.jpg
.....4307_0.jpg
.....2283_0.jpg
.....5793_0.jpg
.....5342_0.jpg
.....2652_0.jpg
.....5307_0.jpg
.....932_0.jpg
.....518_0.jpg
.....6418_0.jpg
.....4492_0.jpg
.....3182_0.jpg
.....4800_0.jpg
.....3553_0.jpg
.....

.....948_0.jpg
.....59_0.jpg
.....2140_0.jpg
.....626_0.jpg
.....2903_0.jpg
.....5450_0.jpg
.....5887_0.jpg
.....2105_0.jpg
.....2946_0.jpg
.....663_0.jpg
.....3140_0.jpg
.....6726_0.jpg
.....1236_0.jpg
.....3903_0.jpg
.....3591_0.jpg
.....6763_0.jpg
.....3105_0.jpg
.....3946_0.jpg
.....1273_0.jpg
.....4887_0.jpg
.....4039_0.jpg
.....3529_0.jpg
.....5039_0.jpg
.....5751_0.jpg
.....527_0.jpg
.....2690_0.jpg
.....5380_0.jpg
.....562_0.jpg
.....5714_0.jpg
.....4380_0.jpg
.....4751_0.jpg
.....3241_0.jpg
.....6427_0.jpg
.....1974_0.jpg
.....1172_0.jpg
.....4714_0.jpg
.....1931_0.jpg
.....6462_0.jpg
.....3204_0.jpg
.....352_0.jpg
.....5124_0.jpg
.....2434_0.jpg
.....783_0.jpg
.....5161_0.jpg
.....317_0.jpg
.....5922_0.jpg
.....2471_0.jpg
.....2_0.jpg
.....6683_0.jpg
.....1393_0.jpg
.....1742_0.jpg
.....4124_0.jpg
.....6252_0.jpg
.....3434_0.jpg
.....4161_0.jpg
.....1707_0.jpg
.....3471_0.jpg
.....6217_0.jpg
.....4922_0.jpg
.....1869_0.jpg
.....4609_0.jpg
.....3319_0.jpg
.....2319_0.jpg
.....

.....2884_0.jpg
.....335_0.jpg
.....5143_0.jpg
.....5900_0.jpg
.....2453_0.jpg
.....5592_0.jpg
.....3884_0.jpg
.....4106_0.jpg
.....1760_0.jpg
.....3416_0.jpg
.....3082_0.jpg
.....4143_0.jpg
.....6235_0.jpg
.....3453_0.jpg
.....4900_0.jpg
.....1008_0.jpg
.....6518_0.jpg
.....4858_0.jpg
.....3999_0.jpg
.....1638_0.jpg
.....5858_0.jpg
.....228_0.jpg
.....86_0.jpg
.....505_0.jpg
.....2263_0.jpg
.....997_0.jpg
.....5736_0.jpg
.....2226_0.jpg
.....191_0.jpg
.....6897_0.jpg
.....1115_0.jpg
.....4773_0.jpg
.....6405_0.jpg
.....3263_0.jpg
.....1956_0.jpg
.....6091_0.jpg
.....1581_0.jpg
.....1150_0.jpg
.....1913_0.jpg
.....6440_0.jpg
.....1539_0.jpg
.....129_0.jpg
.....2162_0.jpg
.....5472_0.jpg
.....604_0.jpg
.....290_0.jpg
.....2127_0.jpg
.....5437_0.jpg
.....2964_0.jpg
.....3127_0.jpg
.....6741_0.jpg
.....3964_0.jpg
.....4437_0.jpg
.....1680_0.jpg
.....6390_0.jpg
.....1069_0.jpg
.....6579_0.jpg
.....479_0.jpg
.....354_0.jpg
.....5961_0.jpg
.....2432_0.jpg
.....785_0.jpg
.....311_0.jpg
.....5

In [42]:
# !mkdir predict_valid_resize
# !mkdir predict_train_resize

# !mkdir predict_train_resize/non-melanoma
# !mkdir predict_train_resize/melanoma
# !mkdir predict_valid_resize/non-melanoma
# !mkdir predict_valid_resize/melanoma

In [43]:
def prepare(filepath, IMG_SIZE):
    img_array = cv2.imread(filepath) 
    res = cv2.resize(img_array, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
#     return np.reshape(res, (3, IMG_SIZE, IMG_SIZE))
    return res

unique_chars = ['melanoma', 'non-melanoma']

for chars in unique_chars:
    for filename in os.listdir("./predict_valid_processed/" + chars):
        destination = './predict_valid_resize/'+ chars + "/" + filename
        
        print(filename + " " + chars)
        
        if (filename == ".ipynb_checkpoints"):
            continue
        else:
            image_data = prepare(('./predict_valid_processed/' + chars + "/" + filename) , 100)
            image = Image.fromarray(image_data)
            try:
                image.save('./predict_valid_resize/' + chars + "/" + filename)
            except:
                os.mkdir('predict_valid_resize/'+chars)
                image.save('./predict_valid_resize/' + chars + "/" + filename)

13819_1.jpg melanoma
040_1.jpg melanoma
02509_1.jpg melanoma
05019_1.jpg melanoma
3043_11.jpg melanoma
12777_1.jpg melanoma
05771_1.jpg melanoma
3807_11.jpg melanoma
12732_1.jpg melanoma
06407_1.jpg melanoma
5242_11.jpg melanoma
5513_11.jpg melanoma
16952_1.jpg melanoma
06093_1.jpg melanoma
16154_1.jpg melanoma
03608_1.jpg melanoma
2427_1.jpg melanoma
05318_1.jpg melanoma
2509_11.jpg melanoma
3427_1.jpg melanoma
1214_11.jpg melanoma
02923_1.jpg melanoma
02160_1.jpg melanoma
15123_1.jpg melanoma
02966_1.jpg melanoma
15960_1.jpg melanoma
14166_1.jpg melanoma
1445_11.jpg melanoma
03160_1.jpg melanoma
16210_1.jpg melanoma
4708_11.jpg melanoma
03125_1.jpg melanoma
01682_1.jpg melanoma
12697_1.jpg melanoma
3819_11.jpg melanoma
12246_1.jpg melanoma
219_1.jpg melanoma
11175_1.jpg melanoma
14713_1.jpg melanoma
16465_1.jpg melanoma
11130_1.jpg melanoma
06975_1.jpg melanoma
1377_11.jpg melanoma
16799_1.jpg melanoma
6210_11.jpg melanoma
03038_1.jpg melanoma
188_11.jpg melanoma
02414_1.jpg melanoma

03980_1.jpg melanoma
16662_1.jpg melanoma
06374_1.jpg melanoma
15551_1.jpg melanoma
4360_11.jpg melanoma
12041_1.jpg melanoma
02186_1.jpg melanoma
02512_1.jpg melanoma
15239_1.jpg melanoma
14239_1.jpg melanoma
24_11.jpg melanoma
11888_1.jpg melanoma
14842_1.jpg melanoma
13511_1.jpg melanoma
01371_1.jpg melanoma
14001_1.jpg melanoma
13983_1.jpg melanoma
06624_1.jpg melanoma
14044_1.jpg melanoma
12983_1.jpg melanoma
12511_1.jpg melanoma
02493_1.jpg melanoma
12554_1.jpg melanoma
4182_11.jpg melanoma
3540_1.jpg melanoma
4481_1.jpg melanoma
263_1.jpg melanoma
2540_1.jpg melanoma
761_11.jpg melanoma
226_1.jpg melanoma
01833_1.jpg melanoma
5299_11.jpg melanoma
04282_1.jpg melanoma
11523_1.jpg melanoma
13655_1.jpg melanoma
12610_1.jpg melanoma
4439_1.jpg melanoma
15345_1.jpg melanoma
3641_1.jpg melanoma
05978_1.jpg melanoma
2641_1.jpg melanoma
15689_1.jpg melanoma
15258_1.jpg melanoma
12748_1.jpg melanoma
16928_1.jpg melanoma
13748_1.jpg melanoma
13065_1.jpg melanoma
2719_1.jpg melanoma
06350_

3767_1.jpg melanoma
4277_1.jpg melanoma
05058_1.jpg melanoma
12773_1.jpg melanoma
15263_1.jpg melanoma
12736_1.jpg melanoma
05730_1.jpg melanoma
11897_1.jpg melanoma
06403_1.jpg melanoma
13773_1.jpg melanoma
11091_1.jpg melanoma
06097_1.jpg melanoma
04730_1.jpg melanoma
13820_1.jpg melanoma
14573_1.jpg melanoma
03575_1.jpg melanoma
06787_1.jpg melanoma
01297_1.jpg melanoma
1351_11.jpg melanoma
04863_1.jpg melanoma
13026_1.jpg melanoma
16640_1.jpg melanoma
12063_1.jpg melanoma
12865_1.jpg melanoma
1251_11.jpg melanoma
3871_1.jpg melanoma
4958_11.jpg melanoma
2871_1.jpg melanoma
16128_1.jpg melanoma
03689_1.jpg melanoma
385_1.jpg melanoma
04399_1.jpg melanoma
06012_1.jpg melanoma
03674_1.jpg melanoma
378_1.jpg melanoma
01502_1.jpg melanoma
04364_1.jpg melanoma
4730_11.jpg melanoma
06057_1.jpg melanoma
11812_1.jpg melanoma
12362_1.jpg melanoma
15959_1.jpg melanoma
3333_1.jpg melanoma
14959_1.jpg melanoma
084_1.jpg melanoma
16229_1.jpg melanoma
2333_1.jpg melanoma
04449_1.jpg melanoma
4850

04979_1.jpg melanoma
126_1.jpg melanoma
163_1.jpg melanoma
06811_1.jpg melanoma
01542_1.jpg melanoma
338_1.jpg melanoma
03634_1.jpg melanoma
06052_1.jpg melanoma
13322_1.jpg melanoma
11817_1.jpg melanoma
06483_1.jpg melanoma
01193_1.jpg melanoma
11011_1.jpg melanoma
01507_1.jpg melanoma
04361_1.jpg melanoma
13367_1.jpg melanoma
1328_11.jpg melanoma
16993_1.jpg melanoma
15632_1.jpg melanoma
05324_1.jpg melanoma
02634_1.jpg melanoma
12322_1.jpg melanoma
1228_11.jpg melanoma
02671_1.jpg melanoma
15919_1.jpg melanoma
05409_1.jpg melanoma
14919_1.jpg melanoma
4821_11.jpg melanoma
11779_1.jpg melanoma
01292_1.jpg melanoma
11784_1.jpg melanoma
16294_1.jpg melanoma
01643_1.jpg melanoma
11355_1.jpg melanoma
1404_11.jpg melanoma
04823_1.jpg melanoma
01606_1.jpg melanoma
1355_11.jpg melanoma
12066_1.jpg melanoma
2219_11.jpg melanoma
05708_1.jpg melanoma
01968_1.jpg melanoma
380_1.jpg melanoma
04708_1.jpg melanoma
3293_1.jpg melanoma
3642_1.jpg melanoma
02428_1.jpg melanoma
6351_11.jpg melanoma
05

12901_1.jpg melanoma
5474_11.jpg melanoma
15885_1.jpg melanoma
13901_1.jpg melanoma
11234_1.jpg melanoma
13142_1.jpg melanoma
14417_1.jpg melanoma
5574_11.jpg melanoma
5225_11.jpg melanoma
2342_11.jpg melanoma
16009_1.jpg melanoma
4446_1.jpg melanoma
12692_1.jpg melanoma
15753_1.jpg melanoma
5409_11.jpg melanoma
15716_1.jpg melanoma
16425_1.jpg melanoma
259_1.jpg melanoma
14382_1.jpg melanoma
1373_11.jpg melanoma
5258_11.jpg melanoma
3608_11.jpg melanoma
4702_1.jpg melanoma
3059_11.jpg melanoma
15225_1.jpg melanoma
4188_1.jpg melanoma
5138_11.jpg melanoma
16582_1.jpg melanoma
14225_1.jpg melanoma
16153_1.jpg melanoma
6830_11.jpg melanoma
1642_11.jpg melanoma
16955_1.jpg melanoma
15967_1.jpg melanoma
4658_1.jpg melanoma
4472_11.jpg melanoma
14967_1.jpg melanoma
2966_11.jpg melanoma
67_11.jpg melanoma
346_1.jpg melanoma
6009_11.jpg melanoma
4973_1.jpg melanoma
4936_1.jpg melanoma
2773_11.jpg melanoma
3244_11.jpg melanoma
15609_1.jpg melanoma
1949_11.jpg melanoma
2610_11.jpg melanoma
1252

...

21909_0.jpg non-melanoma
623_01.jpg non-melanoma
31419_0.jpg non-melanoma
24769_0.jpg non-melanoma
45367_0.jpg non-melanoma
47011_0.jpg non-melanoma
42677_0.jpg non-melanoma
23279_0.jpg non-melanoma
36109_0.jpg non-melanoma
46817_0.jpg non-melanoma
41544_0.jpg non-melanoma
2490_01.jpg non-melanoma
46054_0.jpg non-melanoma
43632_0.jpg non-melanoma
46485_0.jpg non-melanoma
41195_0.jpg non-melanoma
25769_0.jpg non-melanoma
46852_0.jpg non-melanoma
43677_0.jpg non-melanoma
46011_0.jpg non-melanoma
22279_0.jpg non-melanoma
22655_0.jpg non-melanoma
32343_0.jpg non-melanoma
35653_0.jpg non-melanoma
35282_0.jpg non-melanoma
25794_0.jpg non-melanoma
22284_0.jpg non-melanoma
32792_0.jpg non-melanoma
32306_0.jpg non-melanoma
41168_0.jpg non-melanoma
35616_0.jpg non-melanoma
25300_0.jpg non-melanoma
24794_0.jpg non-melanoma
23284_0.jpg non-melanoma
33792_0.jpg non-melanoma
26033_0.jpg non-melanoma
36525_0.jpg non-melanoma
33343_0.jpg non-melanoma
31876_0.jpg non-melanoma
26870_0.jpg non-melanoma
3

45363_0.jpg non-melanoma
47015_0.jpg non-melanoma
42673_0.jpg non-melanoma
6014_01.jpg non-melanoma
1908_0.jpg non-melanoma
41540_0.jpg non-melanoma
46813_0.jpg non-melanoma
46050_0.jpg non-melanoma
43636_0.jpg non-melanoma
6745_01.jpg non-melanoma
46481_0.jpg non-melanoma
3359_01.jpg non-melanoma
1937_01.jpg non-melanoma
41191_0.jpg non-melanoma
44363_0.jpg non-melanoma
41505_0.jpg non-melanoma
43673_0.jpg non-melanoma
46015_0.jpg non-melanoma
41997_0.jpg non-melanoma
42492_0.jpg non-melanoma
45553_0.jpg non-melanoma
42800_0.jpg non-melanoma
33918_0.jpg non-melanoma
42043_0.jpg non-melanoma
42845_0.jpg non-melanoma
31268_0.jpg non-melanoma
6177_01.jpg non-melanoma
6626_01.jpg non-melanoma
36778_0.jpg non-melanoma
44553_0.jpg non-melanoma
41335_0.jpg non-melanoma
32918_0.jpg non-melanoma
46625_0.jpg non-melanoma
19_01.jpg non-melanoma
6077_01.jpg non-melanoma
41370_0.jpg non-melanoma
44516_0.jpg non-melanoma
43845_0.jpg non-melanoma
43006_0.jpg non-melanoma
6726_01.jpg non-melanoma
449

6817_01.jpg non-melanoma
41289_0.jpg non-melanoma
43203_0.jpg non-melanoma
41936_0.jpg non-melanoma
43697_0.jpg non-melanoma
2178_01.jpg non-melanoma
44387_0.jpg non-melanoma
6053_01.jpg non-melanoma
44756_0.jpg non-melanoma
41973_0.jpg non-melanoma
46420_0.jpg non-melanoma
43246_0.jpg non-melanoma
6702_01.jpg non-melanoma
45713_0.jpg non-melanoma
42203_0.jpg non-melanoma
45756_0.jpg non-melanoma
31028_0.jpg non-melanoma
6602_01.jpg non-melanoma
36538_0.jpg non-melanoma
45387_0.jpg non-melanoma
2729_01.jpg non-melanoma
0117_0.jpg non-melanoma
36957_0.jpg non-melanoma
34262_0.jpg non-melanoma
31896_0.jpg non-melanoma
36151_0.jpg non-melanoma
0152_0.jpg non-melanoma
4455_01.jpg non-melanoma
5633_01.jpg non-melanoma
31441_0.jpg non-melanoma
34227_0.jpg non-melanoma
31090_0.jpg non-melanoma
0583_0.jpg non-melanoma
36580_0.jpg non-melanoma
2841_01.jpg non-melanoma
1718_01.jpg non-melanoma
1117_0.jpg non-melanoma
35262_0.jpg non-melanoma
41188_0.jpg non-melanoma
1149_01.jpg non-melanoma
1618

26310_0.jpg non-melanoma
33866_0.jpg non-melanoma
21645_0.jpg non-melanoma
31353_0.jpg non-melanoma
36643_0.jpg non-melanoma
33025_0.jpg non-melanoma
0640_0.jpg non-melanoma
26355_0.jpg non-melanoma
23533_0.jpg non-melanoma
0291_0.jpg non-melanoma
36292_0.jpg non-melanoma
26784_0.jpg non-melanoma
21294_0.jpg non-melanoma
31782_0.jpg non-melanoma
21168_0.jpg non-melanoma
45300_0.jpg non-melanoma
33708_0.jpg non-melanoma
42284_0.jpg non-melanoma
45794_0.jpg non-melanoma
42655_0.jpg non-melanoma
0128_0.jpg non-melanoma
6196_01.jpg non-melanoma
35218_0.jpg non-melanoma
46835_0.jpg non-melanoma
44300_0.jpg non-melanoma
41566_0.jpg non-melanoma
43610_0.jpg non-melanoma
46076_0.jpg non-melanoma
44345_0.jpg non-melanoma
46870_0.jpg non-melanoma
46033_0.jpg non-melanoma
1128_0.jpg non-melanoma
43284_0.jpg non-melanoma
44794_0.jpg non-melanoma
0718_0.jpg non-melanoma
24938_0.jpg non-melanoma
42065_0.jpg non-melanoma
26699_0.jpg non-melanoma
21389_0.jpg non-melanoma
45530_0.jpg non-melanoma
42863

35922_0.jpg non-melanoma
3924_01.jpg non-melanoma
25432_0.jpg non-melanoma
35124_0.jpg non-melanoma
22961_0.jpg non-melanoma
32434_0.jpg non-melanoma
22122_0.jpg non-melanoma
1680_0.jpg non-melanoma
32649_0.jpg non-melanoma
43751_0.jpg non-melanoma
41427_0.jpg non-melanoma
46974_0.jpg non-melanoma
35359_0.jpg non-melanoma
43380_0.jpg non-melanoma
32298_0.jpg non-melanoma
46172_0.jpg non-melanoma
43714_0.jpg non-melanoma
3382_01.jpg non-melanoma
506_01.jpg non-melanoma
41462_0.jpg non-melanoma
34788_0.jpg non-melanoma
45690_0.jpg non-melanoma
42380_0.jpg non-melanoma
33298_0.jpg non-melanoma
33649_0.jpg non-melanoma
45241_0.jpg non-melanoma
34359_0.jpg non-melanoma
479_0.jpg non-melanoma
45204_0.jpg non-melanoma
0744_0.jpg non-melanoma
33121_0.jpg non-melanoma
36747_0.jpg non-melanoma
42039_0.jpg non-melanoma
45529_0.jpg non-melanoma
34431_0.jpg non-melanoma
31257_0.jpg non-melanoma
31686_0.jpg non-melanoma
36396_0.jpg non-melanoma
0395_0.jpg non-melanoma
36702_0.jpg non-melanoma
33164_

In [44]:
for chars in unique_chars:
    for filename in os.listdir("./predict_train_processed/" + chars):
        destination = './predict_train_resize/'+ chars + "/" + filename
        
        print(filename + " " + chars)
        
        if (filename == ".ipynb_checkpoints"):
            continue
        else:
            image_data = prepare(('./predict_train_processed/' + chars + "/" + filename) , 100)
            image = Image.fromarray(image_data)
            try:
                image.save('./predict_train_resize/' + chars + "/" + filename)
            except:
                os.mkdir('predict_train_resize/'+chars)
                image.save('./predict_train_resize/' + chars + "/" + filename)

45678_1.jpg melanoma
3143_11.jpg melanoma
36416_1.jpg melanoma
4860_11.jpg melanoma
21882_1.jpg melanoma
36884_1.jpg melanoma
570_11.jpg melanoma
33235_1.jpg melanoma
36453_1.jpg melanoma
31900_1.jpg melanoma
36082_1.jpg melanoma
26594_1.jpg melanoma
35760_1.jpg melanoma
43368_1.jpg melanoma
32270_1.jpg melanoma
196_11.jpg melanoma
35725_1.jpg melanoma
2092_11.jpg melanoma
41818_1.jpg melanoma
4236_1.jpg melanoma
22723_1.jpg melanoma
41776_1.jpg melanoma
44110_1.jpg melanoma
43400_1.jpg melanoma
44953_1.jpg melanoma
22808_1.jpg melanoma
44155_1.jpg melanoma
41733_1.jpg melanoma
4393_11.jpg melanoma
44916_1.jpg melanoma
43445_1.jpg melanoma
1338_1.jpg melanoma
34008_1.jpg melanoma
21378_1.jpg melanoma
45110_1.jpg melanoma
42400_1.jpg melanoma
26668_1.jpg melanoma
45584_1.jpg melanoma
23808_1.jpg melanoma
0338_1.jpg melanoma
4293_11.jpg melanoma
26201_1.jpg melanoma
45579_1.jpg melanoma
24177_1.jpg melanoma
3389_11.jpg melanoma
26695_1.jpg melanoma
36383_1.jpg melanoma
0380_1.jpg melanom

44405_1.jpg melanoma
5217_11.jpg melanoma
0668_1.jpg melanoma
23989_1.jpg melanoma
24250_1.jpg melanoma
1918_11.jpg melanoma
23740_1.jpg melanoma
36430_1.jpg melanoma
33687_1.jpg melanoma
34397_1.jpg melanoma
24681_1.jpg melanoma
55_11.jpg melanoma
26920_1.jpg melanoma
0476_1.jpg melanoma
5933_11.jpg melanoma
35397_1.jpg melanoma
25681_1.jpg melanoma
3276_11.jpg melanoma
1818_11.jpg melanoma
35746_1.jpg melanoma
32213_1.jpg melanoma
1476_1.jpg melanoma
5626_11.jpg melanoma
44136_1.jpg melanoma
46240_1.jpg melanoma
5177_11.jpg melanoma
46691_1.jpg melanoma
2854_11.jpg melanoma
31299_1.jpg melanoma
5077_11.jpg melanoma
45975_1.jpg melanoma
1965_11.jpg melanoma
23090_1.jpg melanoma
33157_1.jpg melanoma
23441_1.jpg melanoma
24151_1.jpg melanoma
31221_1.jpg melanoma
36774_1.jpg melanoma
33112_1.jpg melanoma
32157_1.jpg melanoma
25912_1.jpg melanoma
46629_1.jpg melanoma
4111_1.jpg melanoma
43849_1.jpg melanoma
25114_1.jpg melanoma
35402_1.jpg melanoma
22896_1.jpg melanoma
41080_1.jpg melanom

42052_1.jpg melanoma
16192_1.jpg melanoma
45193_1.jpg melanoma
44995_1.jpg melanoma
12360_1.jpg melanoma
22419_1.jpg melanoma
43017_1.jpg melanoma
46634_1.jpg melanoma
35835_1.jpg melanoma
22833_1.jpg melanoma
35076_1.jpg melanoma
3205_11.jpg melanoma
1797_1.jpg melanoma
35870_1.jpg melanoma
22035_1.jpg melanoma
167_11.jpg melanoma
25525_1.jpg melanoma
26616_1.jpg melanoma
23070_1.jpg melanoma
33566_1.jpg melanoma
34076_1.jpg melanoma
13709_1.jpg melanoma
736_11.jpg melanoma
45968_1.jpg melanoma
34870_1.jpg melanoma
34033_1.jpg melanoma
21343_1.jpg melanoma
31284_1.jpg melanoma
0797_1.jpg melanoma
13822_1.jpg melanoma
16607_1.jpg melanoma
16293_1.jpg melanoma
11783_1.jpg melanoma
16642_1.jpg melanoma
45643_1.jpg melanoma
43316_1.jpg melanoma
12822_1.jpg melanoma
44606_1.jpg melanoma
46535_1.jpg melanoma
4684_11.jpg melanoma
44643_1.jpg melanoma
1298_11.jpg melanoma
44292_1.jpg melanoma
43782_1.jpg melanoma
14304_1.jpg melanoma
45875_1.jpg melanoma
42526_1.jpg melanoma
15304_1.jpg melan

32643_1.jpg melanoma
25294_1.jpg melanoma
35782_1.jpg melanoma
41468_1.jpg melanoma
3562_11.jpg melanoma
22310_1.jpg melanoma
34782_1.jpg melanoma
10_11.jpg melanoma
24645_1.jpg melanoma
31535_1.jpg melanoma
436_1.jpg melanoma
21860_1.jpg melanoma
24600_1.jpg melanoma
31570_1.jpg melanoma
33606_1.jpg melanoma
473_1.jpg melanoma
3462_11.jpg melanoma
2155_11.jpg melanoma
385_11.jpg melanoma
22520_1.jpg melanoma
5915_11.jpg melanoma
12688_1.jpg melanoma
35563_1.jpg melanoma
5815_11.jpg melanoma
31340_1.jpg melanoma
24030_1.jpg melanoma
33036_1.jpg melanoma
36650_1.jpg melanoma
24873_1.jpg melanoma
11929_1.jpg melanoma
73_11.jpg melanoma
26303_1.jpg melanoma
33073_1.jpg melanoma
4237_11.jpg melanoma
16357_1.jpg melanoma
42646_1.jpg melanoma
4566_11.jpg melanoma
11296_1.jpg melanoma
2972_11.jpg melanoma
45787_1.jpg melanoma
16312_1.jpg melanoma
44787_1.jpg melanoma
15862_1.jpg melanoma
4466_11.jpg melanoma
43646_1.jpg melanoma
46020_1.jpg melanoma
43603_1.jpg melanoma
46449_1.jpg melanoma
5

1411_1.jpg melanoma
6416_11.jpg melanoma
35721_1.jpg melanoma
35049_1.jpg melanoma
44151_1.jpg melanoma
43441_1.jpg melanoma
32559_1.jpg melanoma
43090_1.jpg melanoma
42896_1.jpg melanoma
23849_1.jpg melanoma
45114_1.jpg melanoma
34498_1.jpg melanoma
45912_1.jpg melanoma
33559_1.jpg melanoma
26629_1.jpg melanoma
45290_1.jpg melanoma
42351_1.jpg melanoma
43314_1.jpg melanoma
32249_1.jpg melanoma
46537_1.jpg melanoma
35759_1.jpg melanoma
35388_1.jpg melanoma
44290_1.jpg melanoma
32564_1.jpg melanoma
36302_1.jpg melanoma
21304_1.jpg melanoma
21341_1.jpg melanoma
24527_1.jpg melanoma
6567_11.jpg melanoma
36796_1.jpg melanoma
26280_1.jpg melanoma
34458_1.jpg melanoma
42813_1.jpg melanoma
34089_1.jpg melanoma
43015_1.jpg melanoma
43481_1.jpg melanoma
44191_1.jpg melanoma
35089_1.jpg melanoma
35375_1.jpg melanoma
25663_1.jpg melanoma
46958_1.jpg melanoma
25626_1.jpg melanoma
31513_1.jpg melanoma
410_1.jpg melanoma
33665_1.jpg melanoma
36497_1.jpg melanoma
21491_1.jpg melanoma
2360_11.jpg mela

...

1161_01.jpg non-melanoma
4926_0.jpg non-melanoma
3475_0.jpg non-melanoma
12309_0.jpg non-melanoma
25160_0.jpg non-melanoma
02971_0.jpg non-melanoma
15134_0.jpg non-melanoma
05422_0.jpg non-melanoma
02132_0.jpg non-melanoma
12424_0.jpg non-melanoma
68_01.jpg non-melanoma
3789_0.jpg non-melanoma
15171_0.jpg non-melanoma
05467_0.jpg non-melanoma
02934_0.jpg non-melanoma
6757_01.jpg non-melanoma
6006_01.jpg non-melanoma
15932_0.jpg non-melanoma
25608_0.jpg non-melanoma
12461_0.jpg non-melanoma
1925_01.jpg non-melanoma
06385_0.jpg non-melanoma
16693_0.jpg non-melanoma
2789_0.jpg non-melanoma
11383_0.jpg non-melanoma
01695_0.jpg non-melanoma
14134_0.jpg non-melanoma
11752_0.jpg non-melanoma
04422_0.jpg non-melanoma
01244_0.jpg non-melanoma
03971_0.jpg non-melanoma
21868_0.jpg non-melanoma
14977_0.jpg non-melanoma
03132_0.jpg non-melanoma
06754_0.jpg non-melanoma
13424_0.jpg non-melanoma
16242_0.jpg non-melanoma
1825_01.jpg non-melanoma
6657_01.jpg non-melanoma
03934_0.jpg non-melanoma
11717_

04869_0.jpg non-melanoma
26970_0.jpg non-melanoma
23755_0.jpg non-melanoma
036_0.jpg non-melanoma
1157_01.jpg non-melanoma
12069_0.jpg non-melanoma
3715_0.jpg non-melanoma
4205_0.jpg non-melanoma
22710_0.jpg non-melanoma
6874_01.jpg non-melanoma
25245_0.jpg non-melanoma
3750_0.jpg non-melanoma
4240_0.jpg non-melanoma
22755_0.jpg non-melanoma
22384_0.jpg non-melanoma
25694_0.jpg non-melanoma
3381_0.jpg non-melanoma
938_01.jpg non-melanoma
2160_0.jpg non-melanoma
24475_0.jpg non-melanoma
21213_0.jpg non-melanoma
23926_0.jpg non-melanoma
16839_0.jpg non-melanoma
2923_0.jpg non-melanoma
292_0.jpg non-melanoma
14798_0.jpg non-melanoma
13288_0.jpg non-melanoma
23963_0.jpg non-melanoma
2125_0.jpg non-melanoma
21256_0.jpg non-melanoma
06529_0.jpg non-melanoma
26746_0.jpg non-melanoma
01039_0.jpg non-melanoma
23120_0.jpg non-melanoma
14349_0.jpg non-melanoma
22926_0.jpg non-melanoma
3160_0.jpg non-melanoma
838_01.jpg non-melanoma
1383_01.jpg non-melanoma
22165_0.jpg non-melanoma
4470_0.jpg non-

12478_0.jpg non-melanoma
6878_01.jpg non-melanoma
5070_01.jpg non-melanoma
4547_01.jpg non-melanoma
4614_0.jpg non-melanoma
15168_0.jpg non-melanoma
22301_0.jpg non-melanoma
22795_0.jpg non-melanoma
25654_0.jpg non-melanoma
3341_0.jpg non-melanoma
4651_0.jpg non-melanoma
22344_0.jpg non-melanoma
02968_0.jpg non-melanoma
6978_01.jpg non-melanoma
4316_01.jpg non-melanoma
24611_0.jpg non-melanoma
06708_0.jpg non-melanoma
2304_0.jpg non-melanoma
14168_0.jpg non-melanoma
23301_0.jpg non-melanoma
21834_0.jpg non-melanoma
5170_01.jpg non-melanoma
4447_01.jpg non-melanoma
21032_0.jpg non-melanoma
2341_0.jpg non-melanoma
21871_0.jpg non-melanoma
03968_0.jpg non-melanoma
26522_0.jpg non-melanoma
23795_0.jpg non-melanoma
24285_0.jpg non-melanoma
5934_01.jpg non-melanoma
2746_01.jpg non-melanoma
3271_01.jpg non-melanoma
11825_0.jpg non-melanoma
16576_0.jpg non-melanoma
13310_0.jpg non-melanoma
06060_0.jpg non-melanoma
24179_0.jpg non-melanoma
23469_0.jpg non-melanoma
04316_0.jpg non-melanoma
11066

25597_0.jpg non-melanoma
22087_0.jpg non-melanoma
25905_0.jpg non-melanoma
4143_0.jpg non-melanoma
05329_0.jpg non-melanoma
1405_01.jpg non-melanoma
25146_0.jpg non-melanoma
4900_0.jpg non-melanoma
05404_0.jpg non-melanoma
15112_0.jpg non-melanoma
15951_0.jpg non-melanoma
02114_0.jpg non-melanoma
05896_0.jpg non-melanoma
3498_01.jpg non-melanoma
05441_0.jpg non-melanoma
15157_0.jpg non-melanoma
02151_0.jpg non-melanoma
15914_0.jpg non-melanoma
12096_0.jpg non-melanoma
05090_0.jpg non-melanoma
15586_0.jpg non-melanoma
3598_01.jpg non-melanoma
04896_0.jpg non-melanoma
13890_0.jpg non-melanoma
11774_0.jpg non-melanoma
14112_0.jpg non-melanoma
6184_01.jpg non-melanoma
13402_0.jpg non-melanoma
06772_0.jpg non-melanoma
03114_0.jpg non-melanoma
14951_0.jpg non-melanoma
13096_0.jpg non-melanoma
21499_0.jpg non-melanoma
03580_0.jpg non-melanoma
2549_01.jpg non-melanoma
26189_0.jpg non-melanoma
04090_0.jpg non-melanoma
14586_0.jpg non-melanoma
04441_0.jpg non-melanoma
26558_0.jpg non-melanoma
01

11160_0.jpg non-melanoma
14706_0.jpg non-melanoma
01476_0.jpg non-melanoma
3056_01.jpg non-melanoma
183_01.jpg non-melanoma
3707_01.jpg non-melanoma
11923_0.jpg non-melanoma
06166_0.jpg non-melanoma
21619_0.jpg non-melanoma
16470_0.jpg non-melanoma
13216_0.jpg non-melanoma
188_0.jpg non-melanoma
14482_0.jpg non-melanoma
13192_0.jpg non-melanoma
03484_0.jpg non-melanoma
14810_0.jpg non-melanoma
16325_0.jpg non-melanoma
13543_0.jpg non-melanoma
06633_0.jpg non-melanoma
03055_0.jpg non-melanoma
345_01.jpg non-melanoma
3290_01.jpg non-melanoma
04545_0.jpg non-melanoma
11635_0.jpg non-melanoma
14053_0.jpg non-melanoma
03816_0.jpg non-melanoma
13994_0.jpg non-melanoma
13506_0.jpg non-melanoma
16360_0.jpg non-melanoma
03010_0.jpg non-melanoma
21109_0.jpg non-melanoma
03853_0.jpg non-melanoma
04500_0.jpg non-melanoma
26419_0.jpg non-melanoma
01366_0.jpg non-melanoma
14016_0.jpg non-melanoma
11670_0.jpg non-melanoma
514_01.jpg non-melanoma
02055_0.jpg non-melanoma
02816_0.jpg non-melanoma
05545

In [59]:
# Loading in training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True
)

In [60]:
train_generator = train_datagen.flow_from_directory(
        './predict_train_resize',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

Found 25650 images belonging to 2 classes.


In [61]:
# Loading in testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        './predict_valid_resize',
        target_size=(100, 100),
        batch_size=32,
        class_mode='categorical')

Found 17052 images belonging to 2 classes.


In [62]:
# initialising sequential model and adding layers to it
# cnn = tf.keras.models.Sequential()
# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=(3, 350, 350)))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
# cnn.add(tf.keras.layers.Dense(64, activation='relu'))
# cnn.add(tf.keras.layers.Dense(8, activation='softmax'))

# Initializing sequential model that makes use of convolutional, pooling, and dense layers
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', input_shape=(100, 100, 3), padding='same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = 'same'))
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Flatten())
# cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(2, activation='softmax'))

In [63]:
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
# sess = tf.Session(config=config)

In [64]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 5)       140       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 5)         0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 5)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 25, 25, 48)        2208      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 48)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 48)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2352)             

In [65]:
# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, epochs=3)

Epoch 1/3
802/802 [==============================] - 128s 160ms/step - loss: 0.2988 - accuracy: 0.8640 - val_loss: 0.2678 - val_accuracy: 0.8776
Epoch 2/3
802/802 [==============================] - 120s 149ms/step - loss: 0.1882 - accuracy: 0.9204 - val_loss: 0.1853 - val_accuracy: 0.9219
Epoch 3/3
802/802 [==============================] - 199s 248ms/step - loss: 0.1614 - accuracy: 0.9336 - val_loss: 0.1871 - val_accuracy: 0.9226


In [66]:
# !mkdir predicted_model
cnn.save('predicted_model/my_model')

INFO:tensorflow:Assets written to: predicted_model/my_model/assets


In [67]:
CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath) 
    res = cv2.resize(img_array, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
#     return np.reshape(res, (3, IMG_SIZE, IMG_SIZE))
    return res

path, dirs, files = next(os.walk("./data/test"))
file_count = len(files)

predictions = []
prediction_vals = []
final = []
count = 0
l = os.listdir("./data/test")
random.shuffle(l)
for file in l:
    print(file)
    if (file != ".ipynb_checkpoints"):
        image = prepare('data/test/' + str(file)).tolist()
        #print(image.shape)
        prediction = cnn.predict([image])
        prediction_vals.append(prediction)
        if (prediction[0][0] < prediction[0][1]):
            predicted_result = "non-melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"
        else:
            predicted_result = "melanoma"
            if (file.endswith('0.jpg')):
                final_result = "non-melanoma"
            else:
                final_result = "melanoma"

        if (predicted_result == final_result):
            count += 1
        predictions.append(predicted_result)
        final.append(final_result)
    
print(count)
print(file_count-1)

33_1.jpg
544_0.jpg
426_0.jpg
107_1.jpg
21_1.jpg
148_1.jpg
448_0.jpg
452_0.jpg
206_1.jpg
252_1.jpg
172_1.jpg
409_0.jpg
400_0.jpg
136_1.jpg
9_1.jpg
364_0.jpg
28_1.jpg
214_1.jpg
357_0.jpg
124_1.jpg
474_0.jpg
130_1.jpg
112_1.jpg
583_0.jpg
19_1.jpg
116_1.jpg
520_0.jpg
169_1.jpg
225_1.jpg
587_0.jpg
248_1.jpg
257_1.jpg
226_1.jpg
375_0.jpg
363_0.jpg
506_0.jpg
415_0.jpg
1_1.jpg
239_1.jpg
100_1.jpg
326_0.jpg
228_1.jpg
473_0.jpg
115_1.jpg
78_1.jpg
469_0.jpg
512_0.jpg
62_1.jpg
344_0.jpg
122_1.jpg
501_0.jpg
325_0.jpg
90_1.jpg
518_0.jpg
440_0.jpg
328_0.jpg
76_1.jpg
586_0.jpg
552_0.jpg
229_1.jpg
390_0.jpg
476_0.jpg
416_0.jpg
39_1.jpg
571_0.jpg
593_0.jpg
453_0.jpg
197_1.jpg
199_1.jpg
507_0.jpg
551_0.jpg
264_1.jpg
369_0.jpg
195_1.jpg
502_0.jpg
235_1.jpg
594_0.jpg
154_1.jpg
337_0.jpg
486_0.jpg
457_0.jpg
373_0.jpg
352_0.jpg
234_1.jpg
404_0.jpg
370_0.jpg
178_1.jpg
424_0.jpg
111_1.jpg
274_1.jpg
102_1.jpg
294_1.jpg
515_0.jpg
204_1.jpg
538_0.jpg
79_1.jpg
511_0.jpg
496_0.jpg
189_1.jpg
101_1.jpg
434_0.jpg
303_